## Building Predictive models

In [1]:
import os
import numpy as np
import pandas as pd

### Import data

In [2]:
# path of the processed data
processed_data_path = os.path.join(os.path.pardir,'data','processed')
train_data_path = os.path.join(processed_data_path,'train.csv')
test_data_path = os.path.join(processed_data_path,'test.csv')

In [3]:
train_df = pd.read_csv(train_data_path,index_col='PassengerId')
test_df = pd.read_csv(test_data_path,index_col='PassengerId')

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 33 columns):
Survived                   891 non-null int64
Age                        891 non-null float64
Fare                       891 non-null float64
FamilySize                 891 non-null int64
IsMother                   891 non-null int64
IsMale                     891 non-null int64
Deck_A                     891 non-null int64
Deck_B                     891 non-null int64
Deck_C                     891 non-null int64
Deck_D                     891 non-null int64
Deck_E                     891 non-null int64
Deck_F                     891 non-null int64
Deck_G                     891 non-null int64
Deck_z                     891 non-null int64
Pclass_1                   891 non-null int64
Pclass_2                   891 non-null int64
Pclass_3                   891 non-null int64
Title_Lady                 891 non-null int64
Title_Master               891 non-null int64
Title_Miss       

In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 32 columns):
Age                        418 non-null float64
Fare                       418 non-null float64
FamilySize                 418 non-null int64
IsMother                   418 non-null int64
IsMale                     418 non-null int64
Deck_A                     418 non-null int64
Deck_B                     418 non-null int64
Deck_C                     418 non-null int64
Deck_D                     418 non-null int64
Deck_E                     418 non-null int64
Deck_F                     418 non-null int64
Deck_G                     418 non-null int64
Deck_z                     418 non-null int64
Pclass_1                   418 non-null int64
Pclass_2                   418 non-null int64
Pclass_3                   418 non-null int64
Title_Lady                 418 non-null int64
Title_Master               418 non-null int64
Title_Miss                 418 non-null int64
Title_Mr      

### Data Preparation

In [6]:
# creating the matrix
# here we considered all the rows and all columns beginning from Age thus we did not consider Survived here
# the value is then converted to float
X = train_df.loc[:,'Age':].as_matrix().astype('float')
y = train_df['Survived'].ravel()

upper case letter for matrix and lower case letter for 1D array

In [7]:
print(X.shape,y.shape)

(891, 32) (891,)


###### Train test Split

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(712, 32) (712,)
(179, 32) (179,)


In [9]:
# survival in train and test
print('mean survival in train: {0:.3f}'.format(np.mean(y_train)))
print('mean survival in test: {0:.3f}'.format(np.mean(y_test)))

mean survival in train: 0.383
mean survival in test: 0.385


###### Check scikit version

In [10]:
import sklearn

In [11]:
sklearn.__version__

'0.19.1'

### Baseline Model

In [12]:
from sklearn.dummy import DummyClassifier 

In [13]:
# create model
model_dummy = DummyClassifier(strategy='most_frequent',random_state=0)

In [14]:
# train model
model_dummy.fit(X_train,y_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [15]:
# obtaining the score
print('score for baseline model: {0:.2f}'.format(model_dummy.score(X_test,y_test)))

score for baseline model: 0.61


In [16]:
# performance metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [17]:
# accuracy score which is same as baseline model score
print('accuracy for baseline model: {0:.2f}'.format(accuracy_score(y_test,model_dummy.predict(X_test))))

accuracy for baseline model: 0.61


In [18]:
print('confusion matrix for baseline model: \n {0}'.format(confusion_matrix(y_test,model_dummy.predict(X_test))))

confusion matrix for baseline model: 
 [[110   0]
 [ 69   0]]


In [19]:
print('precision for baseline model: {0:.2f}'.format(precision_score(y_test,model_dummy.predict(X_test))))

precision for baseline model: 0.00


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
print('Recall for baseline model: {0:.2f}'.format(recall_score(y_test,model_dummy.predict(X_test))))

Recall for baseline model: 0.00


#### First Kaggle submission

In [21]:
# preparing the actual test data
test_X = test_df.as_matrix().astype('float')

In [22]:
predictions = model_dummy.predict(test_X)

In [23]:
df_submission = pd.DataFrame({'PassengerId':test_df.index,'Survived' : predictions})

In [24]:
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [25]:
submission_data_path = os.path.join(os.path.pardir,'data','external')
submission_file_path = os.path.join(submission_data_path,'01_dummy.csv')

In [26]:
df_submission.to_csv(submission_file_path,index=False)

In [27]:
def get_submission_file(model,filename):
    test_X = test_df.as_matrix().astype('float')
    predictions = model.predict(test_X)
    df_submission = pd.DataFrame({'PassengerId': test_df.index,'Survived': predictions})
    submission_data_path = os.path.join(os.path.pardir,'data','external')
    submission_file_path = os.path.join(submission_data_path,filename)
    df_submission.to_csv(submission_file_path,index=False)

In [28]:
get_submission_file(model_dummy,'01_dummy.csv')

#### Logistic Regression

In [29]:
from sklearn.linear_model import LogisticRegression

In [30]:
model_lr_1 = LogisticRegression(random_state=0)

In [31]:
model_lr_1.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [32]:
print('score for logistic regression - version 1: {0:.2f}'.format(model_lr_1.score(X_test,y_test)))

score for logistic regression - version 1: 0.83


In [33]:
print('accuracy for logistic regression model: {0:.2f}'.format(accuracy_score(y_test,model_lr_1.predict(X_test))))
print('confusion matrix for logistic regression model: \n {0}'.format(confusion_matrix(y_test,model_lr_1.predict(X_test))))
print('precision for logistic regression model: {0:.2f}'.format(precision_score(y_test,model_lr_1.predict(X_test))))
print('Recall for logistic regression model: {0:.2f}'.format(recall_score(y_test,model_lr_1.predict(X_test))))

accuracy for logistic regression model: 0.83
confusion matrix for logistic regression model: 
 [[95 15]
 [15 54]]
precision for logistic regression model: 0.78
Recall for logistic regression model: 0.78


In [34]:
# model coefficients
model_lr_1.coef_

array([[-0.0284227 ,  0.00455451, -0.50009082,  0.61781337, -0.81392335,
         0.12845078, -0.17281787, -0.39317839,  0.5215999 ,  1.09941225,
         0.40341215, -0.18345055, -0.30036048,  0.96533485,  0.48256746,
        -0.34483453,  0.28089623,  1.21761334,  0.56363964, -1.4458631 ,
         1.07245527, -0.11273715, -0.47293643,  0.1625566 ,  0.2471695 ,
         0.28009381,  0.41324787,  0.49183527,  0.46198828,  0.14924423,
         0.3728351 ,  0.73023268]])

#### Second Kaggle submission

In [35]:
get_submission_file(model_lr_1,'02_lr.csv')

### Part 2

#### Hyperparameter Optimization

In [36]:
model_lr = LogisticRegression(random_state=0)

In [37]:
from sklearn.model_selection import GridSearchCV

In [38]:
parameters = {'C':[1.0,10.0,50.0,100.0,1000.0], 'penalty' : ['l1','l2']}
clf = GridSearchCV(model_lr,param_grid=parameters,cv=3) # 3-fold cross-validation

In [39]:
clf.fit(X_train,y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [40]:
clf.best_params_

{'C': 1.0, 'penalty': 'l1'}

In [41]:
print('best score : {0:.2f}'.format(clf.best_score_))

best score : 0.83


In [42]:
print('score for logistic regression - version 2: {0:.2f}'.format(clf.score(X_test,y_test)))

score for logistic regression - version 2: 0.83


#### Third Kaggle Submission

In [43]:
get_submission_file(clf,'03_lr.csv')

### Feature normalization and standardization

###### Feature Normalization

In [44]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [45]:
 scaler = MinMaxScaler()

In [46]:
 X_train_scaled = scaler.fit_transform(X_train)

In [47]:
 X_train_scaled[:,0].min(),X_train_scaled[:,0].max()

(0.0, 1.0)

In [48]:
 X_test_scaled = scaler.transform(X_test) # normalize test data

###### Feature Standardization

In [49]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

##### Create model after standardization

In [50]:
model_lr_2 = LogisticRegression()
parameters = {'C':[0.5,1.0,10.0,50.0,100.0,1000.0],'penalty':['l1','l2']}
clf = GridSearchCV(model_lr_2,param_grid=parameters,cv=3)

In [51]:
clf.fit(X_train_scaled,y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.5, 1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [52]:
clf.best_params_

{'C': 0.5, 'penalty': 'l1'}

In [53]:
clf.best_score_

0.8230337078651685

In [54]:
print('score for standardized model - version 2 : {0:.2f}'.format(clf.score(X_test_scaled,y_test)))

score for standardized model - version 2 : 0.84


### Model Persistence

In [55]:
import pickle

In [56]:
model_file_path = os.path.join(os.path.pardir,'models','lr_model.pkl')
scaler_file_path = os.path.join(os.path.pardir,'models','lr_scaler.pkl')

In [57]:
# opening the file in binary format
model_file_pickle = open(model_file_path,'wb')
scaler_file_pickle = open(scaler_file_path,'wb')

In [58]:
pickle.dump(clf,model_file_pickle)
pickle.dump(scaler,scaler_file_pickle)

In [59]:
model_file_pickle.close()
scaler_file_pickle.close()

###### load the persisted file

In [60]:
model_file_pickle = open(model_file_path,'rb')
scaler_file_pickle = open(scaler_file_path,'rb')
clf_loaded = pickle.load(model_file_pickle)
scaler_loaded = pickle.load(scaler_file_pickle)
model_file_pickle.close()
scaler_file_pickle.close()

In [61]:
clf_loaded

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.5, 1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [62]:
scaler_loaded

StandardScaler(copy=True, with_mean=True, with_std=True)

In [63]:
X_test_scaled = scaler_loaded.transform(X_test)
print('score for persisted logistic regression: {0:.2f}'.format(clf_loaded.score(X_test_scaled,y_test)))

score for persisted logistic regression: 0.84
